<a href="https://www.kaggle.com/code/psywarrior/project-abstraction?scriptVersionId=255091454" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# ==============================================================================
# ARC Prize 2025 - Full Submission Template
#
# Author: Sanyam Sanjay Sharma
# Date: August 9, 2025
#
# Description:
# This script serves as a complete, self-contained, and compliant submission
# for the Kaggle ARC Prize 2025 competition. It is designed to run within the
# Kaggle Notebook environment, adhering to all rules, including the 12-hour
# runtime limit and the "no internet access" constraint.
#
# The script includes:
#   1. The foundational "Object Identification" module (`find_objects`).
#   2. A functional Domain-Specific Language (DSL) with core transformations.
#   3. A main solver function (`solve_task`) that now performs a basic search.
#   4. A main execution block that simulates the Kaggle environment.
#
# This template provides the complete structure needed to compete. Our goal is
# to incrementally add intelligence to the `solve_task` function.
# ==============================================================================

import json
import os
from pathlib import Path
import numpy as np
from collections import deque
import time

In [2]:
# ==============================================================================
# SECTION 1: CORE PERCEPTION MODULE (The "Eyes")
# This is the `find_objects` function we developed. It is the foundation of
# our solver's ability to understand the content of a grid.
# ==============================================================================

def find_objects(grid):
    """
    Identifies all distinct, contiguous objects in a grid.
    An "object" is defined as a group of connected cells of the same color that
    are not the background color. The background color is determined by finding
    the most frequent color in the grid.
    """
    if not isinstance(grid, np.ndarray):
        grid = np.array(grid, dtype=int)

    height, width = grid.shape
    
    if grid.size == 0:
        return [] # Handle empty grids

    colors, counts = np.unique(grid, return_counts=True)
    background_color = colors[np.argmax(counts)]

    visited = np.zeros_like(grid, dtype=bool)
    objects = []
    object_id_counter = 1

    for r in range(height):
        for c in range(width):
            if visited[r, c] or grid[r, c] == background_color:
                continue

            obj_color = grid[r, c]
            new_object = {
                'id': object_id_counter, 'color': int(obj_color), 'pixels': [],
                'min_row': r, 'max_row': r, 'min_col': c, 'max_col': c
            }
            
            q = deque([(r, c)])
            visited[r, c] = True
            
            while q:
                row, col = q.popleft()
                new_object['pixels'].append((row, col))
                new_object['min_row'] = min(new_object['min_row'], row)
                new_object['max_row'] = max(new_object['max_row'], row)
                new_object['min_col'] = min(new_object['min_col'], col)
                new_object['max_col'] = max(new_object['max_col'], col)
                
                for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < height and 0 <= nc < width and \
                       not visited[nr, nc] and grid[nr, nc] == obj_color:
                        visited[nr, nc] = True
                        q.append((nr, nc))
            
            # After finding all pixels, add the object's local shape
            obj_h = new_object['max_row'] - new_object['min_row'] + 1
            obj_w = new_object['max_col'] - new_object['min_col'] + 1
            shape = np.full((obj_h, obj_w), int(background_color), dtype=int)
            for r_pix, c_pix in new_object['pixels']:
                shape[r_pix - new_object['min_row'], c_pix - new_object['min_col']] = obj_color
            new_object['shape'] = shape

            objects.append(new_object)
            object_id_counter += 1
            
    return objects

In [3]:
# ==============================================================================
# SECTION 2: DOMAIN-SPECIFIC LANGUAGE (DSL) (The "Hands")
# This section contains our library of functions that can manipulate grids
# and objects. This is the toolbox our solver will use.
# ==============================================================================

def get_background_color(grid):
    """Helper function to determine the background color of a grid."""
    colors, counts = np.unique(grid, return_counts=True)
    return colors[np.argmax(counts)]

def recolor_object(grid, obj, new_color):
    """
    Changes the color of a specific object on the grid.
    Returns a new grid with the modification.
    """
    new_grid = grid.copy()
    for r, c in obj['pixels']:
        new_grid[r, c] = new_color
    return new_grid

def move_object(grid, obj, dr, dy):
    """
    Moves an object by a relative vector (dr, dy).
    It erases the object from its old location and redraws it at the new one.
    Returns a new grid with the modification.
    """
    new_grid = grid.copy()
    background_color = get_background_color(grid)
    
    # Erase the object from its original position
    for r, c in obj['pixels']:
        new_grid[r, c] = background_color
        
    # Draw the object in its new position
    height, width = new_grid.shape
    for r, c in obj['pixels']:
        nr, nc = r + dr, c + dy
        if 0 <= nr < height and 0 <= nc < width:
            new_grid[nr, nc] = obj['color']
            
    return new_grid

def rotate_object(grid, obj, rotation_degrees):
    """
    Rotates an object in place by 90, 180, or 270 degrees.
    """
    if rotation_degrees not in [90, 180, 270]:
        return grid

    k = rotation_degrees // 90
    rotated_shape = np.rot90(obj['shape'], k)
    
    new_grid = grid.copy()
    background_color = get_background_color(grid)

    # Erase the original object
    for r, c in obj['pixels']:
        new_grid[r, c] = background_color

    # Paste the rotated shape
    h, w = rotated_shape.shape
    min_r, min_c = obj['min_row'], obj['min_col']
    grid_h, grid_w = new_grid.shape

    for r in range(h):
        for c in range(w):
            if rotated_shape[r, c] != get_background_color(rotated_shape):
                nr, nc = min_r + r, min_c + c
                if 0 <= nr < grid_h and 0 <= nc < grid_w:
                    new_grid[nr, nc] = rotated_shape[r, c]
    
    return new_grid

In [4]:
# ==============================================================================
# SECTION 3: THE MAIN SOLVER (The "Brain")
# This is the "brain" of our operation. It takes a single task and tries to
# find a solution by searching through possible transformations.
# ==============================================================================

def solve_task(task):
    """
    Analyzes a task's training pairs and attempts to find a single-step
    transformation rule that solves the task. Then applies the rule to the
    test inputs.
    """
    solution_program = None
    
    # --- Step 1: Generate Candidate Programs ---
    # A "program" is a function and its arguments. We will generate a list
    # of all plausible single-step transformations based on the first training example.
    
    first_train_pair = task['train'][0]
    input_grid = np.array(first_train_pair['input'])
    input_objects = find_objects(input_grid)
    
    candidate_programs = []
    
    for obj in input_objects:
        # Candidate 'recolor' programs
        for color in range(10): # Colors are 0-9
            if color != obj['color']:
                candidate_programs.append(('recolor', {'obj_id': obj['id'], 'new_color': color}))
        
        # Candidate 'move' programs (small moves)
        for dr in range(-5, 6):
            for dy in range(-5, 6):
                if dr != 0 or dy != 0:
                    candidate_programs.append(('move', {'obj_id': obj['id'], 'dr': dr, 'dy': dy}))
        
        # Candidate 'rotate' programs
        for angle in [90, 180, 270]:
             candidate_programs.append(('rotate', {'obj_id': obj['id'], 'angle': angle}))

    # --- Step 2: Search for a Valid Program ---
    # We iterate through our candidate programs and test them against ALL training examples.
    
    for program_name, params in candidate_programs:
        is_solution = True
        for train_pair in task['train']:
            train_in = np.array(train_pair['input'])
            train_out = np.array(train_pair['output'])
            
            # Re-find objects for the current grid, as they might change
            current_objects = find_objects(train_in)
            obj_to_transform = next((o for o in current_objects if o['id'] == params['obj_id']), None)

            if not obj_to_transform:
                is_solution = False
                break

            # Apply the transformation
            if program_name == 'recolor':
                transformed_grid = recolor_object(train_in, obj_to_transform, params['new_color'])
            elif program_name == 'move':
                transformed_grid = move_object(train_in, obj_to_transform, params['dr'], params['dy'])
            elif program_name == 'rotate':
                transformed_grid = rotate_object(train_in, obj_to_transform, params['angle'])
            else:
                transformed_grid = train_in

            if not np.array_equal(transformed_grid, train_out):
                is_solution = False
                break
        
        if is_solution:
            solution_program = (program_name, params)
            # print(f"Found solution: {solution_program}") # Uncomment for debugging
            break # Found a solution, no need to search further

    # --- Step 3: Apply the Solution to Test Grids ---
    predictions = []
    for test_pair in task['test']:
        test_input_grid = np.array(test_pair['input'])
        
        if solution_program:
            program_name, params = solution_program
            
            current_objects = find_objects(test_input_grid)
            obj_to_transform = next((o for o in current_objects if o['id'] == params['obj_id']), None)

            if obj_to_transform:
                if program_name == 'recolor':
                    predicted_grid = recolor_object(test_input_grid, obj_to_transform, params['new_color'])
                elif program_name == 'move':
                    predicted_grid = move_object(test_input_grid, obj_to_transform, params['dr'], params['dy'])
                elif program_name == 'rotate':
                    predicted_grid = rotate_object(test_input_grid, obj_to_transform, params['angle'])
                else:
                    predicted_grid = test_input_grid # Fallback
            else:
                predicted_grid = test_input_grid # Fallback if object not found
        else:
            # Baseline prediction if no solution was found
            predicted_grid = test_input_grid
            
        predictions.append(predicted_grid.tolist()) # Convert back to list of lists
        
    return predictions

In [5]:
# ==============================================================================
# SECTION 4: KAGGLE SUBMISSION BOILERPLATE
# This main execution block handles the competition's file I/O requirements.
# ==============================================================================

if __name__ == '__main__':
    start_time = time.time()
    # Define file paths.
    data_path = Path('/kaggle/input/arc-prize-2025')
    test_challenges_file = data_path / 'arc-agi_test_challenges.json'
    submission_file = Path('/kaggle/working/submission.json')

    # If not in the Kaggle environment, create dummy files for local testing.
    if not data_path.exists():
        print("Kaggle environment not found. Creating dummy files for local testing.")
        os.makedirs('/kaggle/input/arc-prize-2025', exist_ok=True)
        os.makedirs('/kaggle/working', exist_ok=True)
        
        dummy_task_id = "6150a2bd" # A simple move task
        dummy_data = {
            dummy_task_id: {
                "train": [{"input": [[0,0,0],[0,5,0],[0,0,0]], "output": [[0,0,0],[0,0,5],[0,0,0]]}],
                "test": [{"input": [[0,2,0],[0,0,0],[0,0,0]], "output": [[0,0,2],[0,0,0],[0,0,0]]}]
            }
        }
        with open(test_challenges_file, 'w') as f:
            json.dump(dummy_data, f)

    # Load the test challenges from the JSON file.
    try:
        with open(test_challenges_file, 'r') as f:
            tasks = json.load(f)
    except FileNotFoundError:
        print(f"Error: Test file not found at {test_challenges_file}")
        tasks = {}

    # Initialize the submission dictionary.
    submission = {}

    # Process each task.
    for i, (task_id, task_data) in enumerate(tasks.items()):
        print(f"Processing task {i+1}/{len(tasks)}: {task_id}")
        
        # Call our main solver function for the current task.
        predicted_grids = solve_task(task_data)
        
        # Format the predictions according to the competition's `submission.json` schema.
        task_predictions = []
        for grid in predicted_grids:
            task_predictions.append({
                "attempt_1": grid,
                "attempt_2": grid # Using the same prediction for both attempts
            })
        
        submission[task_id] = task_predictions

    # Write the final submission dictionary to `submission.json`.
    with open(submission_file, 'w') as f:
        json.dump(submission, f)

    end_time = time.time()
    print("-" * 30)
    print(f"Submission file created at: {submission_file}")
    print(f"Total tasks processed: {len(submission)}")
    print(f"Total runtime: {end_time - start_time:.2f} seconds")
    print("Script finished successfully.")

Processing task 1/240: 00576224
Processing task 2/240: 007bbfb7
Processing task 3/240: 009d5c81
Processing task 4/240: 00d62c1b
Processing task 5/240: 00dbd492
Processing task 6/240: 017c7c7b
Processing task 7/240: 025d127b
Processing task 8/240: 03560426
Processing task 9/240: 045e512c
Processing task 10/240: 0520fde7
Processing task 11/240: 05269061
Processing task 12/240: 05a7bcf2
Processing task 13/240: 05f2a901
Processing task 14/240: 0607ce86
Processing task 15/240: 0692e18c
Processing task 16/240: 06df4c85
Processing task 17/240: 070dd51e
Processing task 18/240: 08ed6ac7
Processing task 19/240: 09629e4f
Processing task 20/240: 0962bcdd
Processing task 21/240: 09c534e7
Processing task 22/240: 0a1d4ef5
Processing task 23/240: 0a2355a6
Processing task 24/240: 0a938d79
Processing task 25/240: 0b148d64
Processing task 26/240: 0b17323b
Processing task 27/240: 0bb8deee
Processing task 28/240: 0becf7df
Processing task 29/240: 0c786b71
Processing task 30/240: 0c9aba6e
Processing task 31/